In [1]:
# Set up the Open AI Gym
import gym
env = gym.make("CartPole-v0")

obs = env.reset()
print(obs)
# env.render()

[ 0.03265044  0.01979864 -0.00648509 -0.04388959]


In [2]:
# Ask the environment what actions are possible (what is the action space)?
env.action_space

Discrete(2)

In [3]:
# Discreate(2) means the possible actions are integers 0 and 1 which, in this case, represent
#  accelerating left (0) or right (1)

# Since the poll is leaning toward the right, let's accelerate the cart toward the right
#  by using the step() method which executes the given action
action = 1
obs, reward, done, info = env.step(action)
print(obs, reward, done, info)
# env.render()

[ 0.03304641  0.21501298 -0.00736288 -0.33861155] 1.0 False {}


In [4]:
# Let's hardcode a simple policy that will
#  - Accelerate left when the pole is leaning toward the left and
#  - Accelerate right when the pole is leaning toward the right

def basic_policy(obs):
    angle = obs[2]
    return 0 if angle < 0 else 1

totals = []
for episode in range(500):
    episode_rewards = 0
    obs = env.reset()
    for step in range(1000):
        action = basic_policy(obs)
        obs, reward, done, info = env.step(action)
        episode_rewards += reward
        if done:
            break
    totals.append(episode_rewards)
    

import numpy as np
np.mean(totals), np.std(totals), np.min(totals), np.max(totals)

(42.036, 8.042307131663152, 24.0, 67.0)